<a href="https://colab.research.google.com/github/HoTrungQuan2004/ParkingAreaManage/blob/main/License_plate_detection_tflite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 17483, done.
remote: Total 17483 (delta 0), reused 0 (delta 0), pack-reused 17483 (from 1)
Receiving objects: 100% (17483/17483), 16.53 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (11988/11988), done.
/content/yolov5/yolov5/ParkingAreaManage/dataset/yolov5


In [8]:
!git clone https://github.com/HoTrungQuan2004/ParkingAreaManage
%cd ParkingAreaManage/dataset

fatal: destination path 'ParkingAreaManage' already exists and is not an empty directory.
/content/yolov5/yolov5/ParkingAreaManage/dataset


In [11]:
#check dataset structure (don't need to run)
!ls /content/yolov5/yolov5/ParkingAreaManage/dataset/images/train

CarLongPlate0_jpg.rf.f4ed10413fc73418cee9fb6c20268114.jpg
CarLongPlate100_jpg.rf.593c96a5425a3cfe51ba5575ec1dc6b3.jpg
CarLongPlate101_jpg.rf.5614a235a9c47b000f85e6d453f5a233.jpg
CarLongPlate104_jpg.rf.523686949c0b33bcc32cb9aacc302937.jpg
CarLongPlate105_jpg.rf.7a51b05300c8fe71c834c80aecd3c306.jpg
CarLongPlate106_jpg.rf.592b724f2917070e2ed602eff0438270.jpg
CarLongPlate107_jpg.rf.0b558638c74f36a0b23128dd90c70151.jpg
CarLongPlate108_jpg.rf.6f46a8754529ad6dbf8ac477d3dfbd12.jpg
CarLongPlate109_jpg.rf.e709e7cfb637b4c86cb09b458a2b6238.jpg
CarLongPlate10_jpg.rf.e3d806b8a1164bd7c4035a6a94638506.jpg
CarLongPlate110_jpg.rf.ef36ee1c4b8cc23c427f60d3ed051feb.jpg
CarLongPlate112_jpg.rf.48509fb17d7dd75c652f543a2887b534.jpg
CarLongPlate114_jpg.rf.e31e374587fe867adad92e31a8a00092.jpg
CarLongPlate117_jpg.rf.912518e2fb8e5560366ebf66945152dd.jpg
CarLongPlate118_jpg.rf.e641595af048a65310e3d261f4ff6ac4.jpg
CarLongPlate119_jpg.rf.c9c8611f35f78d45935ba1a3f6e4453d.jpg
CarLongPlate11_jpg.rf.3793dcacee948b9a624ef

In [17]:
%cd /content/yolov5

/content/yolov5


In [21]:
!python /content/yolov5/yolov5/train.py \
  --img 416 \
  --batch 8 \
  --epochs 50 \
  --data /content/yolov5/yolov5/ParkingAreaManage/dataset/dataset.yaml \
  --weights yolov5s.pt \
  --name plate_detector

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-06-03 16:48:40.784831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748969320.819029   41845 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748969320.829304   41845 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: 10422069 (10422069-vietnamese-german-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=/content/yolov5/yolov5/ParkingAreaManage/dataset/dataset.yaml, hyp=yolov5/data/hyps/hyp.